In [1]:
import os
import json
from web3 import Web3
from pathlib import Path
from dotenv import load_dotenv
import streamlit as st
load_dotenv()


True

In [2]:
contract_address = os.getenv("SMART_CONTRACT_ADDRESS")

In [3]:
print(contract_address)

0x90e83E1352f0d6D8D322ae0BaBf26AB778b768d5


In [4]:
w3 = Web3(Web3.HTTPProvider(os.getenv("WEB3_PROVIDER_URI")))

In [5]:

#@st.cache_resource()
def load_contract(w3):

    # Load the contract ABI
    with open(Path('./contracts/compiled/mem_token_abi.json')) as f:
        mem_token_abi = json.load(f)

    contract_address = os.getenv("SMART_CONTRACT_ADDRESS")

    # Load the contract
    contract = w3.eth.contract(
        address=contract_address,
        abi=mem_token_abi
    )

    return contract



In [6]:
contract = load_contract(w3)

In [7]:
print(contract)

In [ ]:
traits = contract.functions.getMembershipInfo(0).call()

In [ ]:
print(traits)

['Russian River Valley', 1000000000000000000, 100, 0]


In [ ]:
class Trait:
    def __init__(self, name, value, maxSupply, mintedCount):
        self.name = name
        self.value = value
        self.maxSupply = maxSupply
        self.mintedCount = mintedCount

In [ ]:
trait_data = contract.functions.getMembershipInfo(0).call()
name, value, maxSupply, mintedCount = trait_data
trait = Trait(name, value, maxSupply, mintedCount)
print("Trait Data:")
print(f"Name: {trait.name}")
print(f"Value: {trait.value}")
print(f"Max Supply: {trait.maxSupply}")
print(f"Remaining: {trait.maxSupply - trait.mintedCount}")

Trait Data:
Name: Russian River Valley
Value: 1000000000000000000
Max Supply: 100
Remaining: 100


In [48]:
trait_options = []

for ele in list(range(0,5)):
    trait_data = contract.functions.getMembershipInfo(ele).call()
    name, value, maxSupply, mintedCount = trait_data
    trait_form = f' {name} | Value: {value} | Remaining Available: {maxSupply-mintedCount}/{maxSupply}'
    print(trait_form)
    traitIndex = ele

    

 Russian River Valley | Value: 1000000000000000000 | Remaining Available: 97/100
 Napa Valley | Value: 2000000000000000000 | Remaining Available: 100/100
 Oregon | Value: 3000000000000000000 | Remaining Available: 100/100
 Columbia Valley | Value: 4000000000000000000 | Remaining Available: 100/100
 Finger Lakes | Value: 5000000000000000000 | Remaining Available: 99/100


In [51]:
trait_options

[]

In [29]:
address='0x0DeA5005b7585721dd83b82c5f9f61c772129392'
acctbal = w3.eth.getBalance(address)
print(acctbal)

37999999924367411790


In [30]:
test = contract.functions.getMembershipInfo(4).call()[1]
print(test)

5000000000000000000


In [16]:
wall_bal = float(round(w3.fromWei(w3.eth.get_balance(address), "ether"),2))
print(wall_bal)

85.0


In [18]:
trait_price_wei

5000000000000000000

In [19]:
trait_price_eth = w3.fromWei(trait_price_wei, 'ether')
print(trait_price_eth)

5


In [22]:
accounts = w3.eth.accounts[4:]

In [23]:
print(accounts)

['0x0DeA5005b7585721dd83b82c5f9f61c772129392', '0xb9575455E6339A97D10D00E2957832cEF3DDBE59', '0x783bC8d62c25003b724F6EB3e6482D8BC36F32aC', '0x8039511cA043613A0C7a36505E888c0B1c5fa679', '0x08a72D2E1C574F76C61eBFf47c9290dB8bE37CF9', '0xFd86A5584BDf7eBfB0Be82a5fcD9B93bC92171c2']


In [55]:
# Define dictionary to hold formatted trait info & traitIndex values
mem_options = {}

# Initialize empty traitIndex
traitIndex = 0

# Use for loop to call all available traits from the deployed contract
for ele in list(range(0, 5)):
    # Call the membership info
    trait_data = contract.functions.getMembershipInfo(ele).call()

    # Call trait price in wei 
    trait_price_wei = w3.toWei(contract.functions.getMembershipInfo(ele).call()[1], 'wei')
    #Convert price
    trait_price_eth= w3.fromWei(trait_price_wei, 'ether') 

    # Save the values to trait_data
    name, Price, maxSupply, mintedCount = trait_data
   
    # Format the trait_data for display
    trait_form = f' {name} | Price: {trait_price_eth} ETH | Remaining: {maxSupply - mintedCount}/{maxSupply}'
    # Add the trait_form and the ele (traitIndex) to the mem_options dictionary 
    mem_options[trait_form] = ele

# Customer selects from the list of regions
# region = st.selectbox("Choose your membership:", options=list(mem_options.keys()))
region = mem_options.keys()

# Update traitIndex based on the selected option
traitIndex = mem_options[region]

# Now traitIndex holds the selected value
# st.write(f"Selected Membership: {traitIndex}")

TypeError: unhashable type: 'dict_keys'

In [44]:
print(mem_options.keys())

dict_keys([' Russian River Valley | Price: 1 ETH | Remaining: 100/100', ' Napa Valley | Price: 2 ETH | Remaining: 100/100', ' Oregon | Price: 3 ETH | Remaining: 100/100', ' Columbia Valley | Price: 4 ETH | Remaining: 100/100', ' Finger Lakes | Price: 5 ETH | Remaining: 99/100'])


In [45]:
traitIndex

0

In [46]:
tx_hash = contract.functions.mintMembership(
    address,
    traitIndex
).transact({'from': address, 'gas': 1000000, 'value': trait_price_wei})
receipt = w3.eth.waitForTransactionReceipt(tx_hash)
st.write(f"Purchase Complete!")

st.write("Here is your receipt:")
st.write(dict(receipt))
